In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import random
from tabulate import tabulate
from time import perf_counter
from sklearn.model_selection import train_test_split 
from sklearn.datasets import fetch_openml

2024-11-25 20:47:32.546998: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-25 20:47:33.177196: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#Paso 1

def get_datos():
    datos=[[  2.01666708 ,  56.18031474], [  4.79734083 ,  47.18848199], [  9.23784581 ,  57.68974048], [ 14.11529384 ,  43.70348368],
         [ 14.92688637 ,  59.10244323], [ 17.34408196 ,  65.96080804], [ 17.62435324 ,  45.74334603], [ 22.41875021 ,  13.575581  ],
         [ 25.3139145  ,  68.43756969], [ 34.85886672 , 147.15375307], [ 38.87476262 ,  25.39687009], [ 42.01380169 ,  74.39010069],
         [ 46.63551059 ,  98.93395801], [ 49.58578273 , 116.07013679], [ 50.18371003 , 138.55546747], [ 52.06630172 , 139.36601894],
         [ 54.68810274 , 150.09622546], [ 57.13046193 , 156.14375739], [ 66.35609935 , 119.75844452], [ 69.05499042 , 139.08155228],
         [ 69.51252436 , 128.72247348], [ 69.83788756 , 152.65110462], [ 79.76649746 , 148.23106977], [ 81.83730552 , 137.86314926],
         [ 87.09879038 , 217.28932067], [ 89.00469759 , 168.64994509], [ 93.17139213 , 163.10598352], [ 93.66070686 , 200.47638924],
         [ 94.1944751  , 150.44019156], [ 97.36920633 , 173.2055957 ]]

    return datos


datos=np.array(get_datos())

x_entrenamiento=datos[:,0].reshape(-1,1)
y_entrenamiento=datos[:,1].reshape(-1,1)


In [3]:
#Paso 2
def compile_fit(capas,x,y,epochs):
    np.random.seed(5)
    tf.random.set_seed(5)
    random.seed(5)  
    
    model=Sequential()
    for index,neuronas_capa in enumerate(capas):
        if (index==0):
            model.add(Dense(neuronas_capa, activation='relu',input_dim=x.shape[1]))
        elif (index==len(capas)-1):
            model.add(Dense(neuronas_capa, activation='linear'))
        else:
            model.add(Dense(neuronas_capa, activation='relu'))
    model.compile(loss='mean_squared_error')
    
    history=model.fit(x, y,epochs=epochs,verbose=False) 

    return model,history

capas=[5,10,3,10,1]
epochs=5000
model,history=compile_fit(capas,x_entrenamiento,y_entrenamiento,epochs)



/home/logongas/Documentos/desarrollo/python_default_env/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-25 20:47:34.186536: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
#Paso 3

x_paso3=np.array([ 2.01666708, 4.79734083, 9.23784581, 14.11529384 ]).reshape(-1,1)
y_true_paso3=np.array([ 56.18031474, 47.18848199, 57.68974048, 43.70348368]).reshape(-1,1)
y_pred_paso3=model.predict(x_paso3,verbose=False)
y_error_paso3=(y_true_paso3-y_pred_paso3)**2

resultado_paso3=np.column_stack((x_paso3,y_true_paso3,y_pred_paso3,y_error_paso3))

print(tabulate(resultado_paso3,headers=["x","y_true","y_pred","error"]))


       x    y_true    y_pred     error
--------  --------  --------  --------
 2.01667   56.1803   36.755   377.341
 4.79734   47.1885   41.127    36.7413
 9.23785   57.6897   48.1087   91.7958
14.1153    43.7035   55.7774  145.78


In [5]:
#Paso 4

mse=np.sum(y_error_paso3)/np.size(y_error_paso3)
print(f"Mean Squared Error (MSE)={mse:.3f}")

Mean Squared Error (MSE)=162.915


In [6]:
#Paso 5
media_valores_verdaderos=np.sum(y_true_paso3)/np.size(y_true_paso3)
suma_errores=np.sum((y_true_paso3-y_pred_paso3)**2)
suma_valores_media=np.sum((y_true_paso3-media_valores_verdaderos)**2)
coeficiente_determinacion=1-(suma_errores/suma_valores_media)

print(f"Coeficiente de determinación o R²={coeficiente_determinacion:.3f}")

Coeficiente de determinación o R²=-3.681


In [7]:
#Paso 6
def get_mse(y_true,y_pred):
    y_error=(y_true-y_pred)**2
    mse=np.sum(y_error)/np.size(y_error)

    return mse

def get_coeficiente_determinacion(y_true,y_pred):
    media_valores_verdaderos=np.sum(y_true)/np.size(y_true)
    suma_errores=np.sum((y_true-y_pred)**2)
    suma_valores_media=np.sum((y_true-media_valores_verdaderos)**2)
    coeficiente_determinacion=1-(suma_errores/suma_valores_media)

    return coeficiente_determinacion


def get_metricas_modelo(model,x,y_true):
    y_pred=model.predict(x,verbose=False)
    mse=get_mse(y_true,y_pred)
    coeficiente_determinacion=get_coeficiente_determinacion(y_true,y_pred)

    return mse,coeficiente_determinacion

mse,coeficiente_determinacion=get_metricas_modelo(model,x_entrenamiento,y_entrenamiento)

print(f"Mean Squared Error (MSE)={mse:.3f}")
print(f"Coeficiente de determinación o R²={coeficiente_determinacion:.3f}")


Mean Squared Error (MSE)=720.424
Coeficiente de determinación o R²=0.748


In [8]:
#Paso 7
capas=[20,40,80,40,20,1]
model,history=compile_fit(capas,x_entrenamiento,y_entrenamiento,epochs)
mse,coeficiente_determinacion=get_metricas_modelo(model,x_entrenamiento,y_entrenamiento)

print(f"Mean Squared Error (MSE)={mse:.3f}")
print(f"Coeficiente de determinación o R²={coeficiente_determinacion:.3f}")


Mean Squared Error (MSE)=479.729
Coeficiente de determinación o R²=0.832


In [9]:
#Paso 8

def compile_fit(capas,x,y,epochs,activation):
    np.random.seed(5)
    tf.random.set_seed(5)
    random.seed(5)  
    
    model=Sequential()
    for index,neuronas_capa in enumerate(capas):
        if (index==0):
            model.add(Dense(neuronas_capa, activation=activation,input_dim=x.shape[1]))
        elif (index==len(capas)-1):
            model.add(Dense(neuronas_capa, activation='linear'))
        else:
            model.add(Dense(neuronas_capa, activation=activation))
    model.compile(loss='mean_squared_error')
    
    history=model.fit(x, y,epochs=epochs,verbose=False) 

    return model,history



capas=[20,40,80,40,20,1]
model,history=compile_fit(capas,x_entrenamiento,y_entrenamiento,epochs,"selu")
mse,coeficiente_determinacion=get_metricas_modelo(model,x_entrenamiento,y_entrenamiento)

print(f"Mean Squared Error (MSE)={mse:.3f}")
print(f"Coeficiente de determinación o R²={coeficiente_determinacion:.3f}")


Mean Squared Error (MSE)=214.376
Coeficiente de determinación o R²=0.925


In [10]:
#Paso 9

def compile_fit(capas,x,y,epochs,activation):
    np.random.seed(5)
    tf.random.set_seed(5)
    random.seed(5)  
    
    model=Sequential()
    for index,neuronas_capa in enumerate(capas):
        if (index==0):
            model.add(Dense(neuronas_capa, activation=activation,input_dim=x.shape[1]))
        elif (index==len(capas)-1):
            model.add(Dense(neuronas_capa, activation='linear'))
        else:
            model.add(Dense(neuronas_capa, activation=activation))
    model.compile(loss='mean_squared_error')
    
    tiempo_entrenamiento = perf_counter()
    history=model.fit(x, y,epochs=epochs,verbose=False) 
    tiempo_entrenamiento=perf_counter()-tiempo_entrenamiento

    return model,history,tiempo_entrenamiento

redes_neuronales=[
    [[5,10,30,10,1],"relu"],
    [[5,10,30,10,1],"selu"],
    [[5,10,30,10,1],"tanh"],    
    [[20,40,80,40,20,1],"relu"],
    [[20,40,80,40,20,1],"selu"],
    [[20,40,80,40,20,1],"tanh"],
    [[20,40,80,160,80,40,20,1],"relu"],
    [[20,40,80,160,80,40,20,1],"selu"],
    [[20,40,80,160,80,40,20,1],"tanh"]                           
]
epochs=5000

resultados=[]

for index,(capas,activacion) in enumerate(redes_neuronales):

    model,history,tiempo_entrenamiento=compile_fit(capas,x_entrenamiento,y_entrenamiento,epochs,activacion)
    mse_entrenamiento,coeficiente_determinacion_entrenamiento=get_metricas_modelo(model,x_entrenamiento,y_entrenamiento)

    resultado=[str(index+1),str(capas),epochs,activacion,mse_entrenamiento,coeficiente_determinacion_entrenamiento,tiempo_entrenamiento]
    resultados.append(resultado)

print(tabulate(resultados,headers=["Nombre\nRed","Capas","Épocas","Activación","MSE","R²","Tiempo\nEntrenamiento"]))


  Nombre  Capas                               Épocas  Activación         MSE           R²           Tiempo
     Red                                                                                     Entrenamiento
--------  --------------------------------  --------  ------------  --------  -----------  ---------------
       1  [5, 10, 30, 10, 1]                    5000  relu           699.618   0.754861            62.19
       2  [5, 10, 30, 10, 1]                    5000  selu           565.859   0.801729            62.5378
       3  [5, 10, 30, 10, 1]                    5000  tanh          5790.5    -1.02893             62.5708
       4  [20, 40, 80, 40, 20, 1]               5000  relu           479.729   0.831908            60.9812
       5  [20, 40, 80, 40, 20, 1]               5000  selu           214.376   0.924885            59.9829
       6  [20, 40, 80, 40, 20, 1]               5000  tanh          2869.38   -0.00540068          57.5197
       7  [20, 40, 80, 160, 80, 40, 20,

In [11]:
#Paso 10
def get_datos_validacion():
    datos_validacion=[[  1.22140488 , 59.35315077] , [  2.42834632 ,  3.50613409] , [  4.27529991 , 70.39938914] ,
        [ 14.44651349 , 50.0606769 ] , [ 16.10795855 , 81.08562061] , [ 16.75024181 , 33.95365822] ,
        [ 26.80487149 , 47.1495392 ] , [ 28.81517859 ,106.34919698] , [ 48.56698654 ,120.25398606] ,
        [ 52.08015067 ,116.7993955 ] , [ 53.30646055 ,131.30936472] , [ 55.09968806 ,131.34281752] ,
        [ 60.39615207 , 97.77483743] , [ 73.52487026 , 92.30645543] , [ 76.2771471  ,109.9995226 ] ,
        [ 84.56808303 ,120.60657657] , [ 89.2700557  ,117.3687155 ] , [ 91.03720679 ,159.47376137] ,
        [ 93.53406333 ,166.44439331] , [ 94.78103495 ,180.66942656]]
    return datos_validacion

datos_validacion=np.array(get_datos_validacion())
x_validacion=datos_validacion[:,0].reshape(-1,1)
y_validacion=datos_validacion[:,1].reshape(-1,1)


resultados=[]

for index,(capas,activacion) in enumerate(redes_neuronales):

    model,history,tiempo_entrenamiento=compile_fit(capas,x_entrenamiento,y_entrenamiento,epochs,activacion)
    mse_entrenamiento,coeficiente_determinacion_entrenamiento=get_metricas_modelo(model,x_entrenamiento,y_entrenamiento)
    mse_validacion,coeficiente_determinacion_validacion=get_metricas_modelo(model,x_validacion,y_validacion)   

    resultado=[str(index+1),str(capas),epochs,activacion,mse_entrenamiento,coeficiente_determinacion_entrenamiento,tiempo_entrenamiento,mse_validacion,coeficiente_determinacion_validacion]
    resultados.append(resultado)

print(tabulate(resultados,headers=["Nombre\nRed","Capas","Épocas","Activación","MSE\nEntrenamiento","R²\nEntrenamiento","Tiempo\nEntrenamiento","MSE\nValidación","R²\nValidación"]))



  Nombre  Capas                               Épocas  Activación                MSE               R²           Tiempo           MSE            R²
     Red                                                              Entrenamiento    Entrenamiento    Entrenamiento    Validación    Validación
--------  --------------------------------  --------  ------------  ---------------  ---------------  ---------------  ------------  ------------
       1  [5, 10, 30, 10, 1]                    5000  relu                  699.618       0.754861            62.5292       844.235     0.57673
       2  [5, 10, 30, 10, 1]                    5000  selu                  565.859       0.801729            63.7465       841.22      0.578241
       3  [5, 10, 30, 10, 1]                    5000  tanh                 5790.5        -1.02893             61.2735      3639.78     -0.82486
       4  [20, 40, 80, 40, 20, 1]               5000  relu                  479.729       0.831908            63.8758       998.9

In [12]:
#Paso 11
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import random
from tabulate import tabulate
from time import perf_counter
from sklearn.model_selection import train_test_split 
from sklearn.datasets import fetch_openml

def get_datos():
    datos=[[  2.01666708 ,  56.18031474], [  4.79734083 ,  47.18848199], [  9.23784581 ,  57.68974048], [ 14.11529384 ,  43.70348368],
         [ 14.92688637 ,  59.10244323], [ 17.34408196 ,  65.96080804], [ 17.62435324 ,  45.74334603], [ 22.41875021 ,  13.575581  ],
         [ 25.3139145  ,  68.43756969], [ 34.85886672 , 147.15375307], [ 38.87476262 ,  25.39687009], [ 42.01380169 ,  74.39010069],
         [ 46.63551059 ,  98.93395801], [ 49.58578273 , 116.07013679], [ 50.18371003 , 138.55546747], [ 52.06630172 , 139.36601894],
         [ 54.68810274 , 150.09622546], [ 57.13046193 , 156.14375739], [ 66.35609935 , 119.75844452], [ 69.05499042 , 139.08155228],
         [ 69.51252436 , 128.72247348], [ 69.83788756 , 152.65110462], [ 79.76649746 , 148.23106977], [ 81.83730552 , 137.86314926],
         [ 87.09879038 , 217.28932067], [ 89.00469759 , 168.64994509], [ 93.17139213 , 163.10598352], [ 93.66070686 , 200.47638924],
         [ 94.1944751  , 150.44019156], [ 97.36920633 , 173.2055957 ]]

    return datos

def get_datos_validacion():
    datos_validacion=[[  1.22140488 , 59.35315077] , [  2.42834632 ,  3.50613409] , [  4.27529991 , 70.39938914] ,
        [ 14.44651349 , 50.0606769 ] , [ 16.10795855 , 81.08562061] , [ 16.75024181 , 33.95365822] ,
        [ 26.80487149 , 47.1495392 ] , [ 28.81517859 ,106.34919698] , [ 48.56698654 ,120.25398606] ,
        [ 52.08015067 ,116.7993955 ] , [ 53.30646055 ,131.30936472] , [ 55.09968806 ,131.34281752] ,
        [ 60.39615207 , 97.77483743] , [ 73.52487026 , 92.30645543] , [ 76.2771471  ,109.9995226 ] ,
        [ 84.56808303 ,120.60657657] , [ 89.2700557  ,117.3687155 ] , [ 91.03720679 ,159.47376137] ,
        [ 93.53406333 ,166.44439331] , [ 94.78103495 ,180.66942656]]
    return datos_validacion

def compile_fit(capas,x,y,epochs,activation):
    np.random.seed(5)
    tf.random.set_seed(5)
    random.seed(5)  
    
    model=Sequential()
    for index,neuronas_capa in enumerate(capas):
        if (index==0):
            model.add(Dense(neuronas_capa, activation=activation,input_dim=x.shape[1]))
        elif (index==len(capas)-1):
            model.add(Dense(neuronas_capa, activation='linear'))
        else:
            model.add(Dense(neuronas_capa, activation=activation))
    model.compile(loss='mean_squared_error',metrics=["r2_score"])
    
    tiempo_entrenamiento = perf_counter()
    history=model.fit(x, y,epochs=epochs,verbose=False) 
    tiempo_entrenamiento=perf_counter()-tiempo_entrenamiento

    return model,history,tiempo_entrenamiento

def get_mse(y_true,y_pred):
    y_error=(y_true-y_pred)**2
    mse=np.sum(y_error)/np.size(y_error)

    return mse

def get_coeficiente_determinacion(y_true,y_pred):
    media_valores_verdaderos=np.sum(y_true)/np.size(y_true)
    suma_errores=np.sum((y_true-y_pred)**2)
    suma_valores_media=np.sum((y_true-media_valores_verdaderos)**2)
    coeficiente_determinacion=1-(suma_errores/suma_valores_media)

    return coeficiente_determinacion


def get_metricas_modelo(model,x,y_true):
    y_pred=model.predict(x,verbose=False)
    mse=get_mse(y_true,y_pred)
    coeficiente_determinacion=get_coeficiente_determinacion(y_true,y_pred)

    return mse,coeficiente_determinacion


datos=np.array(get_datos())
x_entrenamiento=datos[:,0].reshape(-1,1)
y_entrenamiento=datos[:,1].reshape(-1,1)

datos_validacion=np.array(get_datos_validacion())
x_validacion=datos_validacion[:,0].reshape(-1,1)
y_validacion=datos_validacion[:,1].reshape(-1,1)


redes_neuronales=[
    [[5,10,30,10,1],"relu"],
    [[5,10,30,10,1],"selu"],
    [[5,10,30,10,1],"tanh"],    
    [[20,40,80,40,20,1],"relu"],
    [[20,40,80,40,20,1],"selu"],
    [[20,40,80,40,20,1],"tanh"],
    [[20,40,80,160,80,40,20,1],"relu"],
    [[20,40,80,160,80,40,20,1],"selu"],
    [[20,40,80,160,80,40,20,1],"tanh"]                           
]
epochs=5000

resultados=[]

for index,(capas,activacion) in enumerate(redes_neuronales):

    model,history,tiempo_entrenamiento=compile_fit(capas,x_entrenamiento,y_entrenamiento,epochs,activacion)
    mse_entrenamiento,coeficiente_determinacion_entrenamiento=get_metricas_modelo(model,x_entrenamiento,y_entrenamiento)
    mse_validacion,coeficiente_determinacion_validacion=get_metricas_modelo(model,x_validacion,y_validacion)   

    resultado=[str(index+1),str(capas),epochs,activacion,mse_entrenamiento,coeficiente_determinacion_entrenamiento,tiempo_entrenamiento,mse_validacion,coeficiente_determinacion_validacion]
    resultados.append(resultado)  

print(tabulate(resultados,headers=["Nombre\nRed","Capas","Épocas","Activación","MSE\nEntrenamiento","R²\nEntrenamiento","Tiempo\nEntrenamiento","MSE\nValidación","R²\nValidación"]))

  Nombre  Capas                               Épocas  Activación                MSE               R²           Tiempo           MSE            R²
     Red                                                              Entrenamiento    Entrenamiento    Entrenamiento    Validación    Validación
--------  --------------------------------  --------  ------------  ---------------  ---------------  ---------------  ------------  ------------
       1  [5, 10, 30, 10, 1]                    5000  relu                  699.618       0.754861            77.1727       844.235     0.57673
       2  [5, 10, 30, 10, 1]                    5000  selu                  565.859       0.801729            76.1871       841.22      0.578241
       3  [5, 10, 30, 10, 1]                    5000  tanh                 5790.5        -1.02893             75.9268      3639.78     -0.82486
       4  [20, 40, 80, 40, 20, 1]               5000  relu                  479.729       0.831908            74.753        998.9

In [13]:
def get_datos():
    boston=fetch_openml(name="boston",version=1)   
    x=np.array(boston.data).astype(np.float32)
    y=np.array(boston.target).astype(np.float32)

    x_entrenamiento, x_validacion, y_entrenamiento, y_validacion = train_test_split(x, y, test_size=0.2, random_state=42)

    return x_entrenamiento, y_entrenamiento.reshape(-1,1), x_validacion, y_validacion.reshape(-1,1)



x_entrenamiento, y_entrenamiento, x_validacion, y_validacion=get_datos()

redes_neuronales=[
    [[20,1],"relu"],
    [[20,1],"selu"],
    [[20,1],"tanh"],       
    [[20,10,1],"relu"],
    [[20,10,1],"selu"],
    [[20,10,1],"tanh"],        
    [[20,30,10,1],"relu"],
    [[20,30,10,1],"selu"],
    [[20,30,10,1],"tanh"],  
    [[20,40,80,40,20,1],"relu"],
    [[20,40,80,40,20,1],"selu"],
    [[20,40,80,40,20,1],"tanh"],
    [[20,40,80,160,80,40,20,1],"relu"],
    [[20,40,80,160,80,40,20,1],"selu"],
    [[20,40,80,160,80,40,20,1],"tanh"]                               
]                     

epochs=1000

resultados=[]



for index,(capas,activacion) in enumerate(redes_neuronales):

    model,history,tiempo_entrenamiento=compile_fit(capas,x_entrenamiento,y_entrenamiento,epochs,activacion)
    mse_entrenamiento,coeficiente_determinacion_entrenamiento=get_metricas_modelo(model,x_entrenamiento,y_entrenamiento)
    mse_validacion,coeficiente_determinacion_validacion=get_metricas_modelo(model,x_validacion,y_validacion)   

    resultado=[str(index+1),str(capas),epochs,activacion,mse_entrenamiento,coeficiente_determinacion_entrenamiento,tiempo_entrenamiento,mse_validacion,coeficiente_determinacion_validacion]
    resultados.append(resultado)  

print(tabulate(resultados,headers=["Nombre\nRed","Capas","Épocas","Activación","MSE\nEntrenamiento","R²\nEntrenamiento","Tiempo\nEntrenamiento","MSE\nValidación","R²\nValidación"]))


/home/logongas/Documentos/desarrollo/python_default_env/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


  Nombre  Capas                               Épocas  Activación                MSE               R²           Tiempo           MSE            R²
     Red                                                              Entrenamiento    Entrenamiento    Entrenamiento    Validación    Validación
--------  --------------------------------  --------  ------------  ---------------  ---------------  ---------------  ------------  ------------
       1  [20, 1]                               1000  relu                 17.6675        0.796629            20.4627       19.3099     0.736684
       2  [20, 1]                               1000  selu                 17.9466        0.793416            20.1527       20.5799     0.719366
       3  [20, 1]                               1000  tanh                 56.1387        0.353787            20.3573       49.3726     0.326742
       4  [20, 10, 1]                           1000  relu                 12.7836        0.852848            20.6068       18.